This notebook is used to train a Tensorflow model to classify audio samples
using the previously saved frequency profiles dataset.

In [2]:
import os
from tensorflow.keras import layers

from micmon.dataset import Dataset
from micmon.model import Model

datasets_dir = os.path.expanduser(os.path.join('~', 'datasets', 'baby-monitor', 'datasets'))
model_path = os.path.expanduser(os.path.join('~', 'models', 'baby-monitor', 'model_test.h5'))
epochs = 2

Load the datasets from the compressed files.

In [3]:
datasets = Dataset.scan(datasets_dir, validation_split=0.3)
freq_bins = len(datasets[0].samples[0])

In [4]:
model = Model(
    [
        layers.Input(shape=(freq_bins,)),
        layers.Dense(int(2.0 * freq_bins), activation='relu'),
        layers.Dense(int(freq_bins), activation='relu'),
        layers.Dense(len(datasets[0].labels), activation='softmax'),
    ],
    labels=['negative', 'positive'],
    low_freq=datasets[0].low_freq,
    high_freq=datasets[0].high_freq,
)

In [5]:
for epoch in range(epochs):
    for i, dataset in enumerate(datasets):
        print(f'[epoch {epoch+1}/{epochs}] [audio sample {i+1}/{len(datasets)}]')
        history = model.fit(dataset)
        evaluation = model.evaluate(dataset)
        print(f'Validation set loss and accuracy: {evaluation}')

[epoch 1/2] [audio sample 1/3]
41/41 [==============================] - 0s 5ms/step - loss: 0.0300 - accuracy: 0.9946
Validation set loss and accuracy: [0.03000183030962944, 0.9945694208145142]
[epoch 1/2] [audio sample 2/3]
47/47 [==============================] - 0s 2ms/step - loss: 0.0353 - accuracy: 0.9926
Validation set loss and accuracy: [0.03526771813631058, 0.9925675392150879]
[epoch 1/2] [audio sample 3/3]
58/58 [==============================] - 0s 7ms/step - loss: 0.1075 - accuracy: 0.9776

Save the model

In [6]:
model.save(model_path, overwrite=True)